In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

pandas objects are equipped with a set of common mathematical and statistical methods.
Most of these fall into the category of reductions or summary statistics, methods
that extract a single value (like the sum or mean) from a Series or a Series of values from
the rows or columns of a DataFrame. Compared with the equivalent methods of vanilla
NumPy arrays, they are all built from the ground up to exclude missing data. Consider
a small DataFrame:

In [2]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5],
[np.nan, np.nan], [0.75, -1.3]],
index=['a', 'b', 'c', 'd'],
columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


Calling DataFrame’s sum method returns a Series containing column sums:

In [3]:
df.sum()

one    9.25
two   -5.80
dtype: float64

Passing axis=1 sums over the rows instead:

In [4]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

NA values are excluded unless the entire slice (row or column in this case) is NA. This
can be disabled using the skipna option:

In [5]:
df.mean(axis=1, skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

Some methods, like idxmin and idxmax, return indirect statistics like the index value
where the minimum or maximum values are attained:

In [7]:
df.idxmax()

one    b
two    d
dtype: object

Other methods are accumulations

In [8]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Another type of method is neither a reduction nor an accumulation. describe is one
such example, producing multiple summary statistics in one shot:

In [9]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


On non-numeric data, describe produces alternate summary statistics

In [10]:
obj = Series(['a', 'a', 'b', 'c'] * 4)
obj

0     a
1     a
2     b
3     c
4     a
5     a
6     b
7     c
8     a
9     a
10    b
11    c
12    a
13    a
14    b
15    c
dtype: object

In [11]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

Method Description<br>
count Number of non-NA values<br>
describe Compute set of summary statistics for Series or each DataFrame column<br>
min, max Compute minimum and maximum values<br>
argmin, argmax Compute index locations (integers) at which minimum or maximum value obtained, respectively<br>
idxmin, idxmax Compute index values at which minimum or maximum value obtained, respectively<br>
quantile Compute sample quantile ranging from 0 to 1<br>
sum Sum of values<br>
mean Mean of values<br>
median Arithmetic median (50% quantile) of values<br>
mad Mean absolute deviation from mean value<br>
var Sample variance of values<br>
std Sample standard deviation of values<br>
skew Sample skewness (3rd moment) of values<br>
kurt Sample kurtosis (4th moment) of values<br>
cumsum Cumulative sum of values<br>
cummin, cummax Cumulative minimum or maximum of values, respectively<br>
cumprod Cumulative product of values<br>
diff Compute 1st arithmetic difference (useful for time series)<br>
pct_change Compute percent changes<br>

<h3>Correlation and Covariance</h3>

In [15]:
import pandas_datareader as pdr

In [20]:
all_data = {}
for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']:
    all_data[ticker] = pdr.get_data_yahoo(ticker, '1/1/2000', '1/1/2010')
    
    price = DataFrame({tic: data['Adj Close'] for tic, data in all_data.items()})
    volume = DataFrame({tic: data['Volume'] for tic, data in all_data.items()})

In [21]:
returns = price.pct_change()

In [22]:
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2009-12-24,0.034340,0.004385,0.002587,0.011117
2009-12-28,0.012294,0.013326,0.005484,0.007098
2009-12-29,-0.011861,-0.003477,0.007058,-0.005571
2009-12-30,0.012147,0.005461,-0.013699,0.005376
2009-12-31,-0.004300,-0.012597,-0.015504,-0.004416


The corr method of Series computes the correlation of the overlapping, non-NA,
aligned-by-index values in two Series. Relatedly, cov computes the covariance:

In [24]:
returns.MSFT.corr(returns.IBM)

0.4959795983674717

In [25]:
returns.MSFT.cov(returns.IBM)

0.0002159577259311431

DataFrame’s corr and cov methods, on the other hand, return a full correlation or
covariance matrix as a DataFrame, respectively:

In [26]:
returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.410011,0.424305,0.470676
IBM,0.410011,1.000000,0.495980,0.390689
MSFT,0.424305,0.495980,1.000000,0.443586
GOOG,0.470676,0.390689,0.443586,1.000000


In [27]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.001027,0.000252,0.000309,0.000303
IBM,0.000252,0.000367,0.000216,0.000142
MSFT,0.000309,0.000216,0.000516,0.000205
GOOG,0.000303,0.000142,0.000205,0.000580


Using DataFrame’s corrwith method, you can compute pairwise correlations between
a DataFrame’s columns or rows with another Series or DataFrame. Passing a Series
returns a Series with the correlation value computed for each column:

In [28]:
returns.corrwith(returns.IBM)

AAPL    0.410011
IBM     1.000000
MSFT    0.495980
GOOG    0.390689
dtype: float64

Passing a DataFrame computes the correlations of matching column names. Here I
compute correlations of percent changes with volume:

In [29]:
returns.corrwith(volume)

AAPL   -0.057549
IBM    -0.007892
MSFT   -0.014245
GOOG    0.062647
dtype: float64

<h3>Unique Values, Value Counts, and Membership</h3>

Another class of related methods extracts information about the values contained in a
one-dimensional Series. To illustrate these, consider this example:

In [30]:
obj = Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

In [31]:
uniques = obj.unique()

In [32]:
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

In [33]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

In [35]:
pd.value_counts(obj.values, sort=False)

d    1
b    2
a    3
c    3
dtype: int64

isin is responsible for vectorized set membership and can be very useful in
filtering a data set down to a subset of values in a Series or column in a DataFrame:

In [36]:
mask = obj.isin(['b', 'c'])

In [37]:
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [38]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

In some cases, you may want to compute a histogram on multiple related columns in
a DataFrame. Here’s an example:

In [39]:
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
'Qu2': [2, 3, 1, 2, 3],
'Qu3': [1, 5, 2, 4, 4]})

In [40]:
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


Passing pandas.value_counts to this DataFrame’s apply function gives:

In [41]:
result = data.apply(pd.value_counts).fillna(0)

In [42]:
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


<h2>Handling Missing Data</h2>

In [2]:
string_data = Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [3]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
string_data[0] = None

In [6]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Argument Description<br>
dropna Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much
missing data to tolerate.<br>
fillna Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill' .<br>
isnull Return like-type object containing boolean values indicating which values are missing / NA.<br>
notnull Negation of isnull .<br>

<h3>Filtering Out Missing Data</h3>

In [7]:
from numpy import nan as NA

In [8]:
data = Series([1, NA, 3.5, NA, 7])

In [9]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [10]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [11]:
data = DataFrame([[1., 6.5, 3.], [1., NA, NA],
[NA, NA, NA], [NA, 6.5, 3.]])

In [12]:
cleaned = data.dropna()

In [13]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [14]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing how='all' will only drop rows that are all NA:

In [15]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Dropping columns in the same way is only a matter of passing axis=1 :

In [16]:
data[4]=NA

In [17]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [18]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose
you want to keep only rows containing a certain number of observations. You can
indicate this with the thresh argument:

In [20]:
df = DataFrame(np.random.randn(7,3))
df

,0,1,2
0,0.530790,1.820979,0.828763
1,-1.466207,1.293301,-2.142601
2,-1.092346,0.888633,-0.016253
3,0.258848,-1.821599,-1.408268
4,-1.039877,-1.537137,0.464077
5,0.418899,0.673003,0.442930
6,0.136677,1.525506,-1.205894


In [23]:
df.loc[:4,1] = NA; df.loc[:2,2]=NA
df

,0,1,2
0,0.530790,NaN,NaN
1,-1.466207,NaN,NaN
2,-1.092346,NaN,NaN
3,0.258848,NaN,-1.408268
4,-1.039877,NaN,0.464077
5,0.418899,0.673003,0.442930
6,0.136677,1.525506,-1.205894


In [24]:
df.dropna(thresh=3)

,0,1,2
5,0.418899,0.673003,0.442930
6,0.136677,1.525506,-1.205894


<h3>Filling in Missing Data</h3>

In [25]:
df.fillna(0)

,0,1,2
0,0.530790,0.000000,0.000000
1,-1.466207,0.000000,0.000000
2,-1.092346,0.000000,0.000000
3,0.258848,0.000000,-1.408268
4,-1.039877,0.000000,0.464077
5,0.418899,0.673003,0.442930
6,0.136677,1.525506,-1.205894


Calling fillna with a dict you can use a different fill value for each column:

In [26]:
df.fillna({1: 0.5, 3:-1})

,0,1,2
0,0.530790,0.500000,NaN
1,-1.466207,0.500000,NaN
2,-1.092346,0.500000,NaN
3,0.258848,0.500000,-1.408268
4,-1.039877,0.500000,0.464077
5,0.418899,0.673003,0.442930
6,0.136677,1.525506,-1.205894


fillna returns a new object, but you can modify the existing object in place:

In [27]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,0.530790,0.000000,0.000000
1,-1.466207,0.000000,0.000000
2,-1.092346,0.000000,0.000000
3,0.258848,0.000000,-1.408268
4,-1.039877,0.000000,0.464077
5,0.418899,0.673003,0.442930
6,0.136677,1.525506,-1.205894


In [29]:
df = DataFrame(np.random.randn(6,3))
df

,0,1,2
0,-0.117709,0.578032,0.570881
1,3.036025,1.187093,-1.118264
2,-1.012357,1.055387,1.206807
3,0.802131,0.539830,-1.437613
4,-0.735336,-2.198529,1.115896
5,-0.254628,-0.027795,-0.367443


In [30]:
df.loc[2:, 1]=NA; df.loc[4:, 2]=NA

In [31]:
df

,0,1,2
0,-0.117709,0.578032,0.570881
1,3.036025,1.187093,-1.118264
2,-1.012357,NaN,1.206807
3,0.802131,NaN,-1.437613
4,-0.735336,NaN,NaN
5,-0.254628,NaN,NaN


In [32]:
df.fillna(method='ffill')

,0,1,2
0,-0.117709,0.578032,0.570881
1,3.036025,1.187093,-1.118264
2,-1.012357,1.187093,1.206807
3,0.802131,1.187093,-1.437613
4,-0.735336,1.187093,-1.437613
5,-0.254628,1.187093,-1.437613


In [33]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.117709,0.578032,0.570881
1,3.036025,1.187093,-1.118264
2,-1.012357,1.187093,1.206807
3,0.802131,1.187093,-1.437613
4,-0.735336,NaN,-1.437613
5,-0.254628,NaN,-1.437613


With fillna you can do lots of other things with a little creativity. For example, you
might pass the mean or median value of a Series:

In [34]:
data = Series([1., NA, 3.5, NA, 7])

In [35]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

<i>fillna function arguments</i><br>
Argument Description<br>
value Scalar value or dict-like object to use to fill missing values<br>
method Interpolation, by default 'ffill' if function called with no other arguments<br>
axis Axis to fill on, default axis=0<br>
inplace Modify the calling object without producing a copy<br>
limit For forward and backward filling, maximum number of consecutive periods to fill<br>